In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\yasba\\OneDrive\\Documents\\Projects\\Audio_Classification_UrbanSound8K'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BasemodelConfig:
    root_dir: Path
    base_model_path: Path

In [5]:
from Audio_Classification.constants import *
from Audio_Classification.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)


    def prepare_base_model_config(self) -> BasemodelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        base_model_config = BasemodelConfig(
            root_dir = config.root_dir,
            base_model_path= config.base_model_path
        )

        return base_model_config

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [8]:
class BaseModel:
    def __init__(self, config: BasemodelConfig):
        self.config = config

    def get_base_model(self):
        self.model = Sequential()

        ## First Layer
        self.model.add(Dense(100,input_shape=(40,)))
        self.model.add(Activation("relu"))
        self.model.add(Dropout(0.5))

        ## Second Layer
        self.model.add(Dense(200))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))

        ## Third Layer
        self.model.add(Dense(100))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))

        ## Final Layer
        self.model.add(Dense(10))
        self.model.add(Activation("softmax"))

        ## Compile the self.model
        self.model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        self.model.summary()

        self.save_model(path=self.config.base_model_path, model=self.model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [9]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.prepare_base_model_config()
    prepare_base_model = BaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
except Exception as e:
    raise e

[2024-12-24 02:07:51,531: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-24 02:07:51,536: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-24 02:07:51,541: INFO: common: created directory at: artifacts/prepare_base_model]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4100      
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 activation_1 (Activation)   (None, 200)             